# Sudoku

Each index is an int 1-9

Each row contains all ints 1-9

Each column contains all ints 1-9

9 Partitioned squares of dimension 3*3 contains all ints 1-9

In [2]:
import pulp as PLP

Model = PLP.LpProblem("Sudoku", PLP.LpMaximize)

sq_row_len = 3
sq_col_len = 3
inputs = [[''],
          ['', 0, 3, 5, 6, 7, 0, 0, 0, 0],
          ['', 4, 0, 0, 8, 2, 9, 5, 0, 0],
          ['', 0, 8, 0, 0, 0, 3, 0, 6, 0],
          ['', 0, 2, 0, 0, 0, 5, 8, 0, 7],
          ['', 8, 0, 0, 2, 0, 6, 0, 0, 5],
          ['', 3, 0, 1, 7, 0, 0, 0, 2, 0],
          ['', 0, 4, 0, 9, 0, 0, 0, 7, 0],
          ['', 0, 0, 2, 4, 8, 7, 0, 0, 6],
          ['', 0, 0, 0, 0, 5, 2, 4, 9, 0]]

assert [len(inputs) == len(inputs[i]) for i in range(1, len(inputs))]

N = len(inputs)-1
ROW = range(1, N+1)
COL = range(1, N+1)
VAL = range(1, N+1)
assert float(N//sq_row_len) == N/sq_row_len
assert float(N//sq_col_len) == N/sq_col_len
assert sq_row_len*sq_col_len == N

x = PLP.LpVariable.dicts("x", (ROW, COL, VAL), lowBound=0, upBound=1, cat=PLP.LpBinary)

# Objekt doesn't matter
Model += PLP.lpSum([x[i][j][v] for i in ROW for j in COL for v in VAL]), "Obj"

# constraints
for i in ROW:
    for j in COL:
        Model += PLP.lpSum([x[i][j][v] for v in VAL]) == 1

for v in VAL:
    for i in ROW:
        Model += PLP.lpSum([x[i][j][v] for j in COL]) == 1
    
    for j in COL:
        Model += PLP.lpSum([x[i][j][v] for i in ROW]) == 1

# square
for v in VAL:
    for offset_i in range(1, N+1, sq_row_len):
        for offset_j in range(1, N+1, sq_col_len):
            Model += PLP.lpSum([x[offset_i + i][offset_j + j][v] for i in range(0, sq_row_len) for j in range(0, sq_col_len)]) == 1

# input
for i in ROW:
    for j in COL:
        value = inputs[i][j]
        if 1 <= value and value <= N:
            Model += x[i][j][value] == 1


#print(Model)
#Model.writeLP("Sudoku.lp")
Model.solve()

#for v in Model.variables():
#    print(v.name, "=", v.varValue)

print("Obj. = ", PLP.value(Model.objective))
print("Status:", PLP.LpStatus[Model.status])

for value in Model.variables():
    type, i, j, v = value.name.split('_')
    if type == 'x':
        i = int(i)
        j = int(j)
        v = int(v)
        if value.varValue == 1:
            inputs[i][j] = v

for i in ROW:
    print([inputs[i][j] for j in COL])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/t_/lny80fx15ylgdtrjm995gln80000gn/T/f3372ae8a1214b6d89babf2f1ca9ba1d-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/t_/lny80fx15ylgdtrjm995gln80000gn/T/f3372ae8a1214b6d89babf2f1ca9ba1d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 365 COLUMNS
At line 5505 RHS
At line 5866 BOUNDS
At line 6596 ENDATA
Problem MODEL has 360 rows, 729 columns and 2952 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 81 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -81 to -1.79769e+308
Probing was tried 0 times a

# Calcudoku

No object function.

Only one value at each index
$$
\sum_{v=1}^{n} x_{i, j, v} = 1
$$
For all $i, j = 1, ..., n$

Only one instance of each value at each row
$$
\sum_{j=1}^{n} x_{i, j, v} = 1
$$
For all $i, v = 1, ..., n$

Only one instance of each value at each column
$$
\sum_{i=1}^{n} x_{i, j, v} = 1
$$
For all $j, v = 1, ..., n$

Inputs
$$
x_{i, j, input_{i, j}} = 1
$$
For all $1 \leq input_{i, j} \leq n$ for all $i, j = 1, ..., n$. 

## Limits +
$$
\sum_{(i, j) \in P} \sum_{v=1}^{n} x_{i, j, v} = R_{P}
$$
For set of points P with result $R_{P}$

## Limit - 
$$
|\sum_{v=1}^{n} x_{P_1, v} \cdot v - \sum_{v=1}^{n} x_{P_2, v}| = R_P
$$

Implies
$$
\sum_{v=1}^{n} x_{P_1, v} \cdot v - \sum_{v=1}^{n} x_{P_2, v} \cdot v = R_{P}(2\delta_{P, 1} - 1)
$$

$$
\sum_{v=1}^{n} x_{P_2, v} \cdot v - \sum_{v=1}^{n} x_{P_1, v} \cdot v = R_{P}(2\delta_{P, 2} - 1)
$$

$$
\delta_{P, 1} + \delta_{P, 2} = 1\\
$$

For two points $P_1, P_2$ with result $R_P$


## Limit *

Constraint
$$
\prod_{(i, j)\in P} (\sum_{v=1}^{n} x_{i, j, v}\cdot v) = R_P
$$
For points P

Take logarithm
$$
\sum_{(i, j)\in P} log(\sum_{v=1}^{n} x_{i, j, v}\cdot v) = log(R_P)
$$

Since all are 0 except one then
$$
\sum_{(i, j)\in P} \sum_{v=1}^{n} x_{i, j, v}\cdot log(v) = log(R_P)
$$

## Limit /

$$
\frac{\sum_{v=1}^{n} x_{P_1, v}*v}{\sum_{v=1}^{n} x_{P_2, v}*v} = R_P \text{ or } \frac{\sum_{v=1}^{n} x_{P_2, v}*v}{\sum_{v=1}^{n} x_{P_1, v}*v} = R_P
$$

For two points $P_1, P_2$ with result $R_P$.

$$
log(\sum_{v=1}^{n} x_{P_1, v}*v) - log(\sum_{v=1}^{n} x_{P_2, v}*v) = log(R_P) \text{ or } log(\sum_{v=1}^{n} x_{P_2, v}*v) - log(\sum_{v=1}^{n} x_{P_2, v}*v) = log(R_P)
$$

$$
|log(\sum_{v=1}^{n} x_{P_1, v}*v) - log(\sum_{v=1}^{n} x_{P_2, v}*v)| = log(R_P)
$$

$$
|\sum_{v=1}^{n} x_{P_1, v}*log(v) - \sum_{v=1}^{n} x_{P_2, v}*log(v)| = log(R_P)
$$

Implies
$$
\sum_{v=1}^{n} x_{P_1, v}*log(v) - \sum_{v=1}^{n} x_{P_2, v}*log(v) = log(R_P) (2\delta_{P_1}-1)
$$

$$
\sum_{v=1}^{n} x_{P_2, v}*log(v) - \sum_{v=1}^{n} x_{P_1, v}*log(v) = log(R_P) (2\delta_{P_2}-1)
$$

$$
\delta_{1} + \delta_{2} = 1
$$

Which is equivalent to subtracting while using a log transformation.

In [5]:
import pulp as PLP
import math

Model = PLP.LpProblem("Calcudoku", PLP.LpMaximize)

INF = 99999
N = 5
limits = [['+', 7, [(1, 1), (1, 2), (2, 2)]],
          ['-', 1, [(2, 1), (3, 1)]],
          ['+', 1, [(3, 2)]],
          ['/', 4, [(4, 1), (4, 2)]],
          ['-', 2, [(5, 1), (5, 2)]],
          ['+', 9, [(1, 3), (2, 3), (3, 3)]],
          ['+', 7, [(4, 3), (4, 4)]],
          ['*', 4, [(5, 3), (5, 4)]],
          ['-', 1, [(4, 5), (5, 5)]],
          ['+', 7, [(3, 4), (3, 5)]],
          ['+', 11, [(2, 4), (2, 5), (1, 5)]],
          ['+', 1, [(1, 4)]]]
inputs = [[0]*(N+1) for i in range(N+1)]
ROW = range(1, N+1)
COL = range(1, N+1)
VAL = range(1, N+1)
assert len(sum([limit[2] for limit in limits], [])) == N*N, 'Not partition'

x = PLP.LpVariable.dicts("x", (ROW, COL, VAL), lowBound=0, upBound=1, cat=PLP.LpBinary)

# create one set of deltas for each minus limit
num_of_minus = sum([1 for limit in limits if limit[0]=='-'])
minus_delta = PLP.LpVariable.dicts('minus_delta', (range(1, num_of_minus+1), [1, 2]), lowBound=0, upBound=1, cat=PLP.LpBinary)

# create one set of deltas for each divide limit
num_of_divide = sum([1 for limit in limits if limit[0]=='/'])
divide_delta = PLP.LpVariable.dicts('divide_delta', (range(1, num_of_divide+1), [1, 2]), lowBound=0, upBound=1, cat=PLP.LpBinary)

# Objekt doesn't matter
Model += PLP.lpSum([x[i][j][v] for i in ROW for j in COL for v in VAL]), "Obj"

# Begrænsninger
for i in ROW:
    for j in COL:
        Model += PLP.lpSum([x[i][j][v] for v in VAL]) == 1

for v in VAL:
    for i in ROW:
        Model += PLP.lpSum([x[i][j][v] for j in COL]) == 1
    
    for j in COL:
        Model += PLP.lpSum([x[i][j][v] for i in ROW]) == 1

# input
for i in ROW:
    for j in COL:
        value = inputs[i][j]
        if 1 <= value and value <= N:
            Model += x[i][j][value] == 1

#
minus_count = 0 # track number of minus limits
divide_count = 0 # track number of divide limits
for limit in limits:
    type = limit[0]
    result = limit[1]
    points = limit[2]
    if type == '+':
        Model += PLP.lpSum([x[i][j][v]*v for (i, j) in points for v in VAL]) == result
    elif type == '-':
        minus_count += 1
        point1 = points[0]
        i1 = point1[0]
        j1 = point1[1]
        point2 = points[1]
        i2 = point2[0]
        j2 = point2[1]
        Model += PLP.lpSum([x[i1][j1][v]*v for v in VAL]) - PLP.lpSum([x[i2][j2][v]*v for v in VAL]) == result*(2*minus_delta[minus_count][1] - 1)
        Model += PLP.lpSum([x[i2][j2][v]*v for v in VAL]) - PLP.lpSum([x[i1][j1][v]*v for v in VAL]) == result*(2*minus_delta[minus_count][2] - 1)
        Model += minus_delta[minus_count][1] + minus_delta[minus_count][2] == 1
        pass
    elif type == '*':
        Model += PLP.lpSum([x[i][j][v]*math.log(v) for v in VAL for (i, j) in points]) == math.log(result)
        pass
    elif type == '/':
        divide_count += 1
        point1 = points[0]
        i1 = point1[0]
        j1 = point1[1]
        point2 = points[1]
        i2 = point2[0]
        j2 = point2[1]
        Model += PLP.lpSum([x[i1][j1][v]*math.log(v) for v in VAL]) - PLP.lpSum([x[i2][j2][v]*math.log(v) for v in VAL]) == math.log(result)*(2*divide_delta[divide_count][1] - 1)
        Model += PLP.lpSum([x[i2][j2][v]*math.log(v) for v in VAL]) - PLP.lpSum([x[i1][j1][v]*math.log(v) for v in VAL]) == math.log(result)*(2*divide_delta[divide_count][2] - 1)
        Model += divide_delta[divide_count][1] + divide_delta[divide_count][2] == 1
        pass 


#print(Model)
Model.solve()


for value in Model.variables():
    type = value.name.split('_')[0]
    if type == 'x':
        i = int(value.name.split('_')[1])
        j = int(value.name.split('_')[2])
        v = int(value.name.split('_')[3])
        if value.varValue == 1:
            inputs[i][j] = v
    else:
        print(value.name + '=' + str(value.varValue))

for i in ROW:
    print([inputs[i][j] for j in COL])

print("Obj. = ", PLP.value(Model.objective))
print("Status:", PLP.LpStatus[Model.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/t_/lny80fx15ylgdtrjm995gln80000gn/T/b1780c01305d4ef3b5f5dc972ee644b0-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/t_/lny80fx15ylgdtrjm995gln80000gn/T/b1780c01305d4ef3b5f5dc972ee644b0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 100 COLUMNS
At line 1042 RHS
At line 1138 BOUNDS
At line 1272 ENDATA
Problem MODEL has 95 rows, 133 columns and 550 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 25 - 0.00 seconds
Cgl0003I 103 fixed, 0 tightened bounds, 0 strengthened rows, 1 substitutions
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root n

# Sudoku with rules

## Diagonal

Every number 1-9 must appear exactly once on each of the two diagonals of the board.

$$
\sum_{(i, j)\in D_k} x_{i, j, v} = 1
$$
For set of diagoal points $D_k$ for $k=1,2$ for all values $v=1,..,9$.

## Non consecutive neighbors

All adjecent numbers must be non-consecutive. A non-consecutive is defined as
$$
|a - b| \geq 2
$$
For any neighbors (up, down, left, right) (a, b)

Must be satisfied
$$
|\sum_{v=1}^{n} x_{P_1, v}*v - \sum_{v=1}^{n} x_{P_2, v}*v| \geq 2
$$

Lp constraint
$$
\sum_{v=1}^{n} x_{P_1, v}*v - \sum_{v=1}^{n} x_{P_2, v}*v - 2 \geq m(1-\delta_{P, 1})
$$

$$
\sum_{v=1}^{n} x_{P_2, v}*v - \sum_{v=1}^{n} x_{P_1, v}*v - 2 \geq m(1-\delta_{P, 2})
$$

$$
\delta_{P, 1} + \delta_{P, 2} = 1
$$
For any two neighbors $P_1, P_2$ and lower bound m.

## Chess king

For a point (i, j), any of the 8 neighbor points can't hold the same value. Since all horizontal and vertical points can't by sudoku definition, it's sufficient to ensure that no diaagonal neighbor contains the same value.

If point contains value then no diagonal neighbors must contain same value.
$$
x_{i, j, v} = 1 \rightarrow \sum_{(a, b)\in P} x_{a, b, v} = 0
$$
Else unconstrained.

$$
\sum_{(a, b)\in P} x_{a, b, v} \leq 4x_{i, j, v}
$$
For set of diagonal neighbor points P for all points (i, j) for all values v.

## Chess Knight

For a point (i, j), any of the 8 knight jump (2 jumps + 1 perpendicular jump) points can't hold the same value.

If point contains value then no knight jump point can contain same value.
$$
x_{i, j, v} = 1 \rightarrow \sum_{(a, b)\in P} x_{a, b, v} = 0
$$
Else unconstrained.

$$
\sum_{(a, b)\in P} x_{a, b, v} \leq 8(1-x_{i, j, v})
$$
For set of knight jump points P for all points (i, j) for all values v.

## Thermometer

A thermometer consists of shaded cells and one cell shaded circular. The circular shade is the smallest number and numbers are strictly increasing until the end of the termometer.

Strict inequality $<$ isn't linear hence we use a weak constraint and add the constraint that the difference between the largest point minus the smallest point must be at least 1.
$$
\sum_{v=1}^{n} x_{P_1, v} v \leq \sum_{v=1}^{n} x_{P_2, v} v
$$

$$
\sum_{v=1}^{n} x_{P_2, v} v - \sum_{v=1}^{n} x_{P_1, v} v \geq 1
$$
For any two consecutive points in order $P_1, P_2$ where $P_1$ contains the smallest number.

## Palindrome

A palindrome consists of shaded cells. The two cells at each end of the shape must be equal, then the two cells length 1 away from the ends must be equal etc. If there's an uneven number of cells in the shape, no rule applies to the middle cell.

$$
\sum_{v=1}^{n} x_{P_1, v} v = \sum_{v=1}^{n} x_{P_2, v} v
$$
For two points $P_1, P_2$ with the same distance to the end.

## Kropki

Adjacent cells with hollow dots must be consecutive and adjacent cells with filled dots must divide and give 2.

Hollow dots are killer constraints where two cells minus to 1 and full dots are killer constraints where two cells divide to 2.

## XV

Two cells separated by X must sum to 10 and two cells separated by V must sum to 5.

X's and V's are killer constraints that sum.

In [7]:
import pulp as PLP
from math import log

Model = PLP.LpProblem("SudokuWithRules", PLP.LpMaximize)

# Rules
diagonals = False
non_consecutive_neighbor = False
chess_king = True
chess_knight = True

# format first point is circle [[(1, 1), (1, 2), (1, 3)], [...]]
thermometer_shapes = []

# format [[(1, 1), (1, 2), (1, 3)], [...]]
palindrome_shapes = [[(2, 4), (3, 3), (4, 2)],
                     [(3, 5), (4, 4), (5, 3)],
                     [(2, 8), (3, 7), (4, 6)],
                     [(6, 4), (7, 3), (8, 2)],
                     [(5, 7), (6, 6), (7, 5)],
                     [(6, 8), (7, 7), (8, 6)]]

# format [['hollow', [(1, 1), (1, 2)]], [...]]
kropki_dots = []

# format [['X', [(1, 1), (1, 2)]], [...]]
xv_dots = []

# format [['*', 10, [(1, 1), (1, 2), (1, 3)]], [...]]
killer_limits = []


inputs = [[''],
          ['', 0, 0, 8, 0, 0, 0, 0, 3, 0],
          ['', 0, 0, 0, 0, 7, 0, 0, 0, 0],
          ['', 4, 0, 0, 0, 0, 0, 0, 0, 8],
          ['', 0, 0, 0, 0, 0, 0, 0, 0, 0],
          ['', 1, 0, 0, 0, 6, 0, 0, 4, 2],
          ['', 0, 0, 0, 0, 0, 0, 0, 0, 0],
          ['', 3, 0, 0, 0, 0, 0, 0, 0, 9],
          ['', 0, 0, 0, 0, 5, 0, 0, 0, 0],
          ['', 0, 0, 4, 0, 0, 0, 3, 0, 0]]

sq_row_len = 3
sq_col_len = 3

assert [len(inputs) == len(inputs[i]) for i in range(1, len(inputs))]

INF = 9999
N = len(inputs)-1
ROW = range(1, N+1)
COL = range(1, N+1)
VAL = range(1, N+1)
assert float(N//sq_row_len) == N/sq_row_len
assert float(N//sq_col_len) == N/sq_col_len
assert sq_row_len*sq_col_len == N

x = PLP.LpVariable.dicts("x", (ROW, COL, VAL), lowBound=0, upBound=1, cat=PLP.LpBinary)

# Object doesn't matter
Model += PLP.lpSum([x[i][j][v] for i in ROW for j in COL for v in VAL]), "Obj"

# Ordinary Sudoku constraints
for i in ROW:
    for j in COL:
        Model += PLP.lpSum([x[i][j][v] for v in VAL]) == 1

for v in VAL:
    for i in ROW:
        Model += PLP.lpSum([x[i][j][v] for j in COL]) == 1
    
    for j in COL:
        Model += PLP.lpSum([x[i][j][v] for i in ROW]) == 1

for v in VAL:
    for offset_i in range(1, N+1, sq_row_len):
        for offset_j in range(1, N+1, sq_col_len):
            Model += PLP.lpSum([x[offset_i + i][offset_j + j][v] for i in range(0, sq_row_len) for j in range(0, sq_col_len)]) == 1

# input
for i in ROW:
    for j in COL:
        value = inputs[i][j]
        if 1 <= value and value <= N:
            Model += x[i][j][value] == 1

# Diagonals
if diagonals:
    diag_set = [[(i, j) for i in ROW for j in COL if i==j],
                [(i, j) for i in ROW for j in COL if i==N+1-j]]
    for v in VAL:
        for k in [0, 1]:
            Model += PLP.lpSum([x[i][j][v] for (i, j) in diag_set[k]]) == 1

# No consecutive neighbor
if non_consecutive_neighbor:
    non_consecutive_delta = PLP.LpVariable.dicts("non_consecutive_delta", (range(1, (N-1)*N*2+1), [1, 2]), lowBound=0, upBound=1, cat=PLP.LpBinary)
    non_consecutive_count = 0
    for i in range(1, N+1):
        for j in range(1, N+1):
            non_consecutive_count += 1
            if j < N:
                Model += PLP.lpSum([x[i][j][v]*v for v in VAL]) - PLP.lpSum([x[i][j+1][v]*v for v in VAL]) - 2 >= -INF*(1-non_consecutive_delta[non_consecutive_count][1])
                Model += PLP.lpSum([x[i][j+1][v]*v for v in VAL]) - PLP.lpSum([x[i][j][v]*v for v in VAL]) - 2 >= -INF*(1-non_consecutive_delta[non_consecutive_count][2])
                Model += non_consecutive_delta[non_consecutive_count][1] + non_consecutive_delta[non_consecutive_count][2] == 1
            elif i < N:
                Model += PLP.lpSum([x[i][j][v]*v for v in VAL]) - PLP.lpSum([x[i+1][j][v]*v for v in VAL]) - 2 >= -INF*(1-non_consecutive_delta[non_consecutive_count][1])
                Model += PLP.lpSum([x[i+1][j][v]*v for v in VAL]) - PLP.lpSum([x[i][j][v]*v for v in VAL]) - 2 >= -INF*(1-non_consecutive_delta[non_consecutive_count][2])
                Model += non_consecutive_delta[non_consecutive_count][1] + non_consecutive_delta[non_consecutive_count][2] == 1

# Chess king
if chess_king:
    for i in ROW:
        for j in COL:
            chess_king_points = [(i+a, j+b) for a in [-1, 1] for b in [-1, 1] if i+a >= 1 and i+a <= N and j+b >= 1 and j+b <= N]
            for v in VAL:
                Model += PLP.lpSum([x[a][b][v] for (a, b) in chess_king_points]) <= 4*(1-x[i][j][v])

# Chess knight
if chess_knight:
    for i in ROW:
        for j in COL:
            chess_knight_points = [(i+a, j+b) for (a, b) in [(1, -2), (-1, -2), (-2, -1), (-2, 1), (-1, 2), (1, 2), (2, 1), (2, -1)] 
                                   if i+a >= 1 and i+a <= N and j+b >= 1 and j+b <= N]
            for v in VAL:
                Model += PLP.lpSum([x[a][b][v] for (a, b) in chess_knight_points]) <= 8*(1-x[i][j][v])

# Thermometer
if len(thermometer_shapes) != 0:
    for shape in thermometer_shapes:
        while len(shape) >= 2:
            p1 = shape[0]
            p2 = shape[1]
            Model += PLP.lpSum([x[p1[0]][p1[1]][v]*v for v in VAL]) <= PLP.lpSum([x[p2[0]][p2[1]][v]*v for v in VAL])
            Model += PLP.lpSum([x[p2[0]][p2[1]][v]*v for v in VAL]) - PLP.lpSum([x[p1[0]][p1[1]][v]*v for v in VAL]) >= 1
            shape.pop(0)

# Palindrome
if len(palindrome_shapes) != 0:
    for shape in palindrome_shapes:
        while len(shape) >= 2:
            p1 = shape[0]
            p2 = shape[-1]
            Model += PLP.lpSum([x[p1[0]][p1[1]][v]*v for v in VAL]) == PLP.lpSum([x[p2[0]][p2[1]][v]*v for v in VAL])
            shape.pop(0)
            shape.pop(-1)

# Kropki
if len(kropki_dots) != 0:
    for type, points in kropki_dots:
        if type == 'hollow':
            killer_limits.append(['-', 1, points])
        elif type == 'full':
            killer_limits.append(['/', 2, points])

# XV
if len(xv_dots) != 0:
    for type, points in xv_dots:
        if type=='X':
            killer_limits.append(['+', 10, points])
        if type=='V':
            killer_limits.append(['+', 5, points])

# killer
if len(killer_limits) != 0:
    num_of_minus = sum([1 for limit in killer_limits if limit[0]=='-'])
    minus_delta = PLP.LpVariable.dicts('minus_delta', (range(1, num_of_minus+1), [1, 2]), lowBound=0, upBound=1, cat=PLP.LpBinary)

    num_of_divide = sum([1 for limit in killer_limits if limit[0]=='/'])
    divide_delta = PLP.LpVariable.dicts('divide_delta', (range(1, num_of_divide+1), [1, 2]), lowBound=0, upBound=1, cat=PLP.LpBinary)
    
    minus_count = 0
    divide_count = 0
    for limit in killer_limits:
        type = limit[0]
        result = limit[1]
        points = limit[2]
        if type == '+':
            Model += PLP.lpSum([x[i][j][v]*v for (i, j) in points for v in VAL]) == result
        elif type == '-':
            minus_count += 1
            point1 = points[0]
            i1 = point1[0]
            j1 = point1[1]
            point2 = points[1]
            i2 = point2[0]
            j2 = point2[1]
            Model += PLP.lpSum([x[i1][j1][v]*v for v in VAL]) - PLP.lpSum([x[i2][j2][v]*v for v in VAL]) == result*(2*minus_delta[minus_count][1] - 1)
            Model += PLP.lpSum([x[i2][j2][v]*v for v in VAL]) - PLP.lpSum([x[i1][j1][v]*v for v in VAL]) == result*(2*minus_delta[minus_count][2] - 1)
            Model += minus_delta[minus_count][1] + minus_delta[minus_count][2] == 1
        elif type == '*':
            Model += PLP.lpSum([x[i][j][v]*log(v) for v in VAL for (i, j) in points]) == log(result)
        elif type == '/':
            divide_count += 1
            point1 = points[0]
            i1 = point1[0]
            j1 = point1[1]
            point2 = points[1]
            i2 = point2[0]
            j2 = point2[1]
            Model += PLP.lpSum([x[i1][j1][v]*log(v) for v in VAL]) - PLP.lpSum([x[i2][j2][v]*log(v) for v in VAL]) == log(result)*(2*divide_delta[divide_count][1] - 1)
            Model += PLP.lpSum([x[i2][j2][v]*log(v) for v in VAL]) - PLP.lpSum([x[i1][j1][v]*log(v) for v in VAL]) == log(result)*(2*divide_delta[divide_count][2] - 1)
            Model += divide_delta[divide_count][1] + divide_delta[divide_count][2] == 1

#print(Model)
Model.solve()

print("Obj. = ", PLP.value(Model.objective))
print("Status:", PLP.LpStatus[Model.status])

for value in Model.variables():
    list_var_name = value.name.split('_')
    if list_var_name[0] == 'x':
        i = int(list_var_name[1])
        j = int(list_var_name[2])
        v = int(list_var_name[3])
        if value.varValue == 1:
            inputs[i][j] = v
    else:
        print(value.name + '=' + str(value.varValue))

for i in ROW:
    print([inputs[i][j] for j in COL])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/t_/lny80fx15ylgdtrjm995gln80000gn/T/9a949ea9cc5c4e74823bfb08f2839341-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/t_/lny80fx15ylgdtrjm995gln80000gn/T/9a949ea9cc5c4e74823bfb08f2839341-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1807 COLUMNS
At line 14827 RHS
At line 16630 BOUNDS
At line 17360 ENDATA
Problem MODEL has 1802 rows, 729 columns and 10832 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 81 - 0.01 seconds
Cgl0003I 280 fixed, 0 tightened bounds, 496 strengthened rows, 14 substitutions
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cut